In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import pandas as pd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

Computer = "SciClone"   # "SciClone"    "WinLap"  "LinLap"   "Desktop"  "PortLap"

#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')
PlotWidthIn = 11
PlotHeightIn = 3.75
PlotDPI = 120

Groups = 1
GroupSize = 15


RollingSize = 25

In [2]:
import multiprocessing
from joblib import Parallel, delayed

import pywt
from pywt._extensions._pywt import (DiscreteContinuousWavelet, ContinuousWavelet,
                                Wavelet, _check_dtype)
from pywt._functions import integrate_wavelet, scale2frequency 

In [3]:
from scipy.optimize import curve_fit

In [4]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'
    

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/CraneData/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'SmallCopy\\'

In [5]:
#Extra Headers:
import os as os
import statistics as st
import os as os
import pandas as pd
import random
import multiprocessing
from joblib import Parallel, delayed
import time

In [ ]:
import CoreFunctions as cf

In [6]:
#Columns names for a file with all 6 dimmensions
Header = np.array(['t', 's','x','y','z','s'])

In [8]:
def get_primelist(upper):
    result=[]
    for cp in range(2,upper+1):
        for i in range(2,cp):
            if cp%i ==0:
                break
        else:
            result.append(cp)
    return result


In [10]:
WaveletToUse = 'gaus2'
#scales = np.linspace(0,2000,1001, dtype=int)
scales = 500
spacer = 10
    


In [17]:
#Inputs:
#folder1 = 'C:\\Users\\Hendrickson\\Desktop\\Phone Acceleration\\'
#folder2 = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'
#folder = "E:\\Documents\\Dan\\Port Work\\OneDrive\OneDrive - The Port of Virginia\\Shared with Everyone\\Crane Data\\"
#folder = "C:\\Users\\Dan\\Desktop\\TempData\\"


files = os.listdir(folder)


In [18]:
#Pre Calculations

num_cores = multiprocessing.cpu_count()-1

if np.shape(files)[0] < GroupSize:
    GroupSize = np.shape(files)[0]

if Groups !=0:
    files = random.sample(files,GroupSize*Groups)

loops = int(float(np.size(files))/float(GroupSize))+1
start = time.time()

inverseRollingSize = 1 / RollingSize

AllAccels = np.asarray([],dtype=object)
for file in files:
    print(file)
    temp = getAcceleration(folder + file)
    if np.size(AllAccels)==0:
        AllAccels = temp
    else:
        AllAccels = np.concatenate((AllAccels,temp), axis =0)

In [19]:
AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(folder+file) for file in files)


In [20]:
MetaData = []  #np.asarray([],dtype=object)
DataOnlyMatrix = np.asarray([],dtype=object)
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False :
        if AllAccels[j][1][4:9] =='Accel':
            print(j,AllAccels[j][1])
    else: 
        for k in range(3):
            MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
            if np.size(DataOnlyMatrix) == 0:
                    DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
            else:
                    DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

MetaData = np.matrix(MetaData)

/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


waveFiltered = np.asarray([],dtype=object)
waveFiltered = Parallel(n_jobs=num_cores)(delayed(low_pass_filter)(np.asarray(data).flatten()) for data in DataOnlyMatrix)
waveFiltered = np.matrix(waveFiltered)


length = np.shape(waveFiltered)[0]
justifier = np.ones((length, np.shape(waveFiltered)[1]))
average = np.zeros(length)
for i in range(length):
    average[i]= np.average(waveFiltered[i][:])
justifier = justifier.T * average.T
waveFiltered = waveFiltered - justifier.T

In [21]:
waveKalmaned = np.asarray([],dtype=object)
waveKalmaned = Parallel(n_jobs=num_cores)(delayed(cf.KalmanFilterDenoise)(np.asarray(data).flatten()) for data in DataOnlyMatrix)
waveKalmaned = np.matrix(waveKalmaned)


/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: 

In [22]:
length = np.shape(waveKalmaned)[0]
justifier = np.ones((length, np.shape(waveKalmaned)[1]))
average = np.zeros(length)
for i in range(length):
    average[i]= np.average(waveKalmaned[i][:])
justifier = justifier.T * average.T
waveKalmaned = waveKalmaned - justifier.T

DataOnlyMatrix = np.matrix(DataOnlyMatrix)

length = np.shape(DataOnlyMatrix)[1]
halfLength = int(length / 2)

Weights1 = np.zeros((halfLength,halfLength))
Weights2 = np.zeros((halfLength,halfLength+RollingSize-1))

for j in range(halfLength):
    r = j+1

    if r < RollingSize:
        ir = 1/ r
        for k in range(r):
            Weights1[k,j] = ir
    else:
        for k in range(RollingSize):
            Weights1[r-RollingSize+k, j] = inverseRollingSize
    for k in range(RollingSize):
        Weights2[j,j+k] = inverseRollingSize

FirstBit = np.dot(DataOnlyMatrix[:,:halfLength], Weights1)
Weights1 = []

if length % 2 == 1: halfLength += 1
Weights2=Weights2[:,:halfLength]
SecondBit = np.dot(DataOnlyMatrix[:,halfLength:], Weights2)

Weights2 = [] 

Weights3 = np.zeros((halfLength,halfLength))
for j in range(RollingSize-1):
    for k in range(RollingSize-j-1):
        Weights3[halfLength-k-1,j] = inverseRollingSize

SecondBit = SecondBit + np.dot(DataOnlyMatrix[:,halfLength:], Weights3)
Weights3 = []

Averages = np.concatenate((FirstBit, SecondBit), axis=1)

In [23]:
j=4
s=0
e=0


length = np.shape(Averages)[0]
justifier = np.ones((length, np.shape(Averages)[1]))
average = np.zeros(length)
for i in range(length):
    average[i]= np.average(Averages[i,:])
justifier = justifier.T * average.T
Averages = Averages - justifier.T
DataOnlyMatrix = DataOnlyMatrix - justifier.T


In [24]:
maxes = np.amax(waveKalmaned[:,500:], axis = 1)
mins = np.amin(waveKalmaned[:,500:], axis = 1)

Keep = np.zeros(mins.size)
for i in range(mins.size):
    if i % 3 == 0:
        if maxes[i] > 0.01 and mins[i] < -0.01:
            Keep[i]=1
            #Keep[i+1]=1
            #Keep[i+2]=1
            print(i)
            

Keep = np.array(Keep, dtype='bool')
            
waveKalmaned = waveKalmaned[Keep,:]
MetaData = MetaData[Keep,:]

0
6
9
12
18
27
30
36
42


In [25]:
if e ==0: e = 60
s=s*1000
e=e*1000
for j in range(np.shape(waveKalmaned)[0]):
    if j %3 ==0:
        fig = plt.figure(figsize=(PlotWidthIn,PlotHeightIn),dpi=PlotDPI)
        #plt.plot(DataOnlyMatrix[j,s:e].T)
        #plt.plot(Averages[j,s:e].T)
        #plt.plot(waveFiltered[j][s:e].T)
        plt.plot(waveKalmaned[j][s:e].T)
        title = str(MetaData[j,0]) + '  ' + str(MetaData[j,1])+ '    ' +str(j)
        plt.title(title)
        plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
        plt.close(fig)

#del DataOnlyMatrix

In [26]:
MetaData[0]

matrix([['220602-0353', 'x', '4', '60kPoints-220602-0353-s4.csv']],
       dtype='<U28')

In [27]:
#FPs = Parallel(n_jobs=num_cores)(delayed(getThumbprint)(motion) for motion in Averages)
#del Averages

#ToPlot=[]
#for i in range(np.shape(FPs)[0]):
#    if MetaData[i][1] == 'x':
#        title = 'FP-'+MetaData[i][0]+'  ' + MetaData[i][1]+ '    ' +str(i)
#        ToPlot.append([FPs[i], title])

In [28]:
#FPs = Parallel(n_jobs=num_cores)(delayed(PlotFingerPrint)(FP) for FP in ToPlot)

In [29]:
WvltFam = pywt.families()
Wvlts = []
for Fam in WvltFam:
    temp = pywt.wavelist(Fam)
    for wvlt in temp:
        Wvlts.append(wvlt)

In [32]:
Bestmove = 6

In [31]:
Bestmove = input('What movement? ')

KeyboardInterrupt: Interrupted by user

In [33]:


Wvlts = np.asarray(Wvlts)
Wvlts = Wvlts.flatten()
#AvgChoosen = np.asarray(Averages[int(Bestmove)]).flatten()
#AvgChoosen = np.asarray(waveFiltered[int(Bestmove)]).flatten()
AvgChoosen = np.asarray(waveKalmaned[int(Bestmove)]).flatten()

data = AvgChoosen
wvt=WaveletToUse
ns=scales
scalespace = spacer
numslices=5
slicethickness=0.12
valleysorpeaks='both'
normconstant=1
plot=False
if np.shape(data)[0] == 2:
    wvt = data[1]
    data = data[0]
#try:
cfX, freqs = cwt_fixed(data, ns, wvt,scalespace)
fp = np.zeros((len(data), ns), dtype=int)

minVal = np.min(cfX)
maxVal = np.max(cfX)
rangeVal = maxVal - minVal#

cfX -= minVal
cfX /= rangeVal

cfX *= float(numslices +1)
cfX = cfX.astype(int)

cfx = np.mod(cfX, 2)


rangeScale = rangeVal / numslices

fp = np.transpose(fp[:,:ns])
H = getThumbprint2(AvgChoosen, 'gaus2')

In [34]:
Allfingers = np.asarray([], dtype=object)

In [37]:
#AllFingers = Parallel(n_jobs=num_cores)(delayed(getThumbprint)([np.asarray(wave).flatten(), 'haar']) for wave in waveKalmaned)


AllFingers = Parallel(n_jobs=num_cores)(delayed(cf.getThumbprint)([AvgChoosen, wave]) for wave in Wvlts[0:25])


/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or

KeyboardInterrupt: 

In [36]:
plots=[]
i=0
for j in range(np.shape(AllFingers)[0]):
    if AllFingers[j] != 'fail':
        plots.append([AllFingers[j], Wvlts[j]])
    else:
        print('fail',i)
    i+=1

NameError: name 'AllFingers' is not defined

In [ ]:
figures = Parallel(n_jobs=num_cores)(delayed(PlotFingerPrint)(FP) for FP in plots)

In [ ]:
#AllCWT = Parallel(n_jobs=num_cores)(delayed(getScalesOnly)([np.asarray(wave).flatten(), 'haar']) for wave in waveKalmaned)


In [ ]:
#np.shape(AllCWT[0][0])

In [ ]:
#np.shape(AllCWT[j])

In [ ]:
#plots=[]
#for j in range(np.shape(AllFingers)[0]):
#    if AllFingers[j] != 'fail':
#        k = PlotColorScales([AllCWT[j], str(MetaData[j,0]) + '  ' + str(MetaData[j,1]) + ' with haar 500x10 full collor'])
#    else:
#        print('fail',i)


### for j in range(np.shape(AllFingers)[0]):
    if j % 3 ==0:
            fig=plt.figure(figsize=(PlotWidthIn,PlotHeightIn),dpi=PlotDPI)
            plt.plot(AllCWT[j][200,:], label = 'Scale 200')
            plt.plot(AllCWT[j][300,:], label = 'Scale 300')
            title = 'CWT Scale for Cut ' + str(MetaData[j,0]) + '  ' + str(MetaData[j,1]) 
            plt.title(title)
            plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
            plt.close(fig)

In [ ]:
def SegmentMove(movement, CheckRange = 750):
    
    DataLength = np.size(movement)
    Segments = np.zeros(DataLength)

    for i in range(DataLength):
        if i < 100:
            cval = 0
        elif i<CheckRange:
            cval = sum(movement[0:i])
        else:
            cval = sum(movement[i-CheckRange:i+CheckRange])
        if cval > 50:
                Segments[i]=2
    return Segments

In [ ]:
#for j in range(np.shape(AllFingers)[0]):
#    if j % 3 ==0:
#            fig=plt.figure(figsize=(PlotWidthIn,PlotHeightIn),dpi=PlotDPI)
#            plt.plot(SegmentMove(AllFingers[j][400,:]), label = 'Segment at Scale 400')
#            #plt.plot(AllFingers[j][200,:], label = 'Scale 200')
#            plt.plot(AllFingers[j][400,:], label = 'Scale 400')
#            fig.legend()
#            title = 'FP Scale 400 Seg Cut ' + str(MetaData[j,0]) + '  ' + str(MetaData[j,1]) 
#            plt.title(title)
#            plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
#            plt.close(fig)
        

In [ ]:
for j in range(np.shape(AllFingers)[0]):
    if j % 3 ==0:
            fig=plt.figure(figsize=(PlotWidthIn,PlotHeightIn),dpi=PlotDPI)
            y = SegmentMove(AllFingers[j][400,:],750)
            plt.plot(y, label = 'Segment at Scale 400')
            #plt.plot(AllFingers[j][200,:], label = 'Scale 200')
            plt.fill_between(np.linspace(0,np.size(y),np.size(y)), y, color='yellow')
            fig.legend()
            title = 'FP Scale 400 Seg fill ' + str(MetaData[j,0]) + '  ' + str(MetaData[j,1]) 
            plt.title(title)
            plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
            plt.close(fig)
        

In [ ]:
#for i in range figures = Parallel(n_jobs=num_cores)(delayed(PlotColorScales)(FP) for FP in plots)

Filters = [[np.asarray(DataOnlyMatrix[int(Bestmove)]).flatten(),'No Filter'],
           [np.asarray(Averages[int(Bestmove)]).flatten(), 'Rolling Average'],
           [np.asarray(waveFiltered[int(Bestmove)]).flatten(), 'Low Pass Filter'],
           [np.asarray(waveKalmaned[int(Bestmove)]).flatten(), 'Kalman filter']]
spacer = 15

Compare = Parallel(n_jobs=num_cores)(delayed(getThumbprint)([signal[0], 'dmey']) for signal in Filters)

plots=[]
i=0
for finger in Compare:
    if finger != 'fail':
        plots.append([finger, Filters[i][1]])
    else:
        print('fail',i)
    i+=1

figures = Parallel(n_jobs=num_cores)(delayed(PlotFingerPrint)(FP) for FP in plots)

for i in range(3):
    fig=plt.figure(figsize=(PlotWidthIn,PlotHeightIn),dpi=PlotDPI)
    plt.plot(Filters[0][0])
    plt.plot(Filters[i][0])
    title = 'Raw Data vs ' + Filters[i][1]
    plt.title(title)
    plt.savefig(location+title.replace(" ", "").replace(":", "").replace(",", "")+FFormat)
    plt.close(fig)